### Restart and Run All Cells

In [1]:
import glob
import os
import pandas as pd
from datetime import date, timedelta
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"
dts_path = "/Users/User/Downloads/Datasets/"

# Directory to be monitored
directory = '/Users/User/Downloads'
# Dictionary of file categories and their extensions
categories = {
    'Images': ['jpeg', 'jpg', 'png'],
    'PDFs': ['pdf'],
    'Datasets': ['csv', 'xlsx', 'xls', 'json'],
    'Videos': ['mp4','m4a'],
    'Zips': ['zip'],
    'EXEs': ['exe', 'msi'],
    'Pythons': ['ipynb'],
}
pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format
year = 2023
month = 8
today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 9, 1), datetime.date(2023, 8, 31))

In [2]:
num_business_days = BDay(2)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2023-09-01
yesterday: 2023-08-30


## Must rerun whenever there is change in orders

In [3]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(47, 8)

In [4]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
input_file

'../data/Price-Today.csv'

In [5]:
prices = pd.read_csv(input_file)
prices.shape

(190, 10)

In [6]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(47, 17)

In [7]:
df_merge['spd'] = 0

In [8]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(47, 12)

In [9]:
file_name = "Price-for-Order.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [10]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [11]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [12]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,BBL,-28,XXX,SET50,400,155.0,169.0,-0.5,-0.294985,0,2023-09-06
B,BEM,-18,XXX,SET50,2500,7.8,8.7,0.05,0.578035,0,2022-12-22
B,BH,-5,XXX,SET50,100,250.0,255.0,-4.0,-1.544402,1,2022-12-22
B,CK,-9,XXX,SET100,1000,22.1,23.0,0.0,0.0,1,2022-12-22
B,CPN,-14,XXX,SET50,300,64.5,68.0,-0.75,-1.090909,0,2022-12-22
B,DELTA,-60,XXX,SET50,200,90.0,110.0,1.5,1.382488,0,2022-12-22
B,DIF,-14,RD40pct,SET,3000,8.65,9.35,-0.1,-1.058201,0,2023-08-15
B,GVREIT,-2,RD20pct,SET,6000,6.9,7.0,-0.05,-0.70922,1,2023-08-28
B,HANA,-12,XXX,SET100,400,59.0,62.0,-0.25,-0.401606,0,2022-12-22
B,ICHI,-22,XXX,SET,1500,14.3,16.5,-0.8,-4.624277,0,2022-12-22
B,III,-15,XXX,SET,2000,10.8,12.3,-0.1,-0.806452,0,2022-12-22
B,IVL,-6,RD30pct,SET50,800,27.5,29.0,0.0,0.0,1,2023-08-28
B,JASIF,-5,RD35pct,SET,4000,6.5,6.75,-0.05,-0.735294,1,2023-08-21
B,JMT,-13,RD25pct,SET50,800,43.0,46.25,-0.25,-0.537634,1,2023-08-24
B,KTB,-6,XXX,SET50,1000,18.8,19.4,0.1,0.518135,

In [13]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\daily'

In [14]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name

df = pd.read_csv(input_file, sep=',', index_col=None)
df['amount'] = df['qty'] * df['target']
df.query('spd >= -3 and spd <= 3')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
7,B,GVREIT,-2,RD20pct,SET,6000,6.90,7.00,-0.05,-0.71,1,2023-08-28,41400.00
30,B,TOA,-2,5pct,SET,1000,26.00,26.50,0.00,0.00,2,2022-12-22,26000.00
33,S,AIMIRT,3,DOS,SET,10000,11.10,10.80,-0.10,-0.92,1,2023-08-23,111000.00


In [15]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
27,B,STA,-8,RD65pct,SET100,2500,14.50,15.30,0.70,4.79,2,2023-08-24,36250.00
30,B,TOA,-2,5pct,SET,1000,26.00,26.50,0.00,0.00,2,2022-12-22,26000.00


In [16]:
df.query('active == 1')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
2,B,BH,-5,XXX,SET50,100,250.00,255.00,-4.00,-1.54,1,2022-12-22,25000.00
3,B,CK,-9,XXX,SET100,1000,22.10,23.00,0.00,0.00,1,2022-12-22,22100.00
7,B,GVREIT,-2,RD20pct,SET,6000,6.90,7.00,-0.05,-0.71,1,2023-08-28,41400.00
11,B,IVL,-6,RD30pct,SET50,800,27.50,29.00,0.00,0.00,1,2023-08-28,22000.00
12,B,JASIF,-5,RD35pct,SET,4000,6.50,6.75,-0.05,-0.74,1,2023-08-21,26000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,S,AWC,39,COS,SET50,9000,5.10,4.26,-0.10,-2.29,1,2022-12-22,45900.00
38,S,CPNREIT,17,COS,SET,5000,12.70,11.00,-0.10,-0.90,1,2023-08-23,63500.00
39,S,JMART,15,5pct,SET50,1200,24.70,23.20,-0.80,-3.33,1,2023-08-24,29640.00
41,S,MCS,17,COS,SET,3000,7.65,6.80,0.00,0.00,1,2023-03-17,22950.00


In [17]:
df.query('name == "JMT"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
13,B,JMT,-13,RD25pct,SET50,800,43.00,46.25,-0.25,-0.54,1,2023-08-24,34400.00


In [18]:
pd.options.display.max_rows = 35
df.query('active == 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,BBL,-28,XXX,SET50,400,155.00,169.00,-0.50,-0.29,0,2023-09-06,62000.00
1,B,BEM,-18,XXX,SET50,2500,7.80,8.70,0.05,0.58,0,2022-12-22,19500.00
4,B,CPN,-14,XXX,SET50,300,64.50,68.00,-0.75,-1.09,0,2022-12-22,19350.00
5,B,DELTA,-60,XXX,SET50,200,90.00,110.00,1.50,1.38,0,2022-12-22,18000.00
6,B,DIF,-14,RD40pct,SET,3000,8.65,9.35,-0.10,-1.06,0,2023-08-15,25950.00
8,B,HANA,-12,XXX,SET100,400,59.00,62.00,-0.25,-0.40,0,2022-12-22,23600.00
9,B,ICHI,-22,XXX,SET,1500,14.30,16.50,-0.80,-4.62,0,2022-12-22,21450.00
10,B,III,-15,XXX,SET,2000,10.80,12.30,-0.10,-0.81,0,2022-12-22,21600.00
16,B,NER,-24,RD40pct,SET,3000,4.44,4.92,0.12,2.50,0,2023-08-25,13320.00
17,B,ORI,-16,RD15pct,SET100,3000,9.40,10.50,-0.10,-0.94,0,2023-08-28,28200.00


In [19]:
df.query('spd <= -40 and trade == "B"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
5,B,DELTA,-60,XXX,SET50,200,90.00,110.00,1.50,1.38,0,2022-12-22,18000.00
19,B,PTTEP,-56,XXX,SET50,200,134.50,162.50,4.00,2.52,0,2023-08-15,26900.00


In [20]:
df.query('trade == "B" & spd > 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [21]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

file_name = "orders-log.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [22]:
df['percent'] = df['%change']

In [23]:
df.query('percent < -2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
9,B,ICHI,-22,XXX,SET,1500,14.30,16.50,-0.80,-4.62,0,2022-12-22,21450.00,-4.62
22,B,SAPPE,-8,XXX,SET,300,89.00,91.00,-9.00,-9.00,1,2022-12-22,26700.00,-9.00
25,B,SINGER,-19,RD50pct,SET100,3000,12.20,14.10,-0.80,-5.37,1,2023-08-03,36600.00,-5.37
26,B,SIRI,-7,XXX,SET,10000,1.90,1.97,-0.07,-3.43,1,2022-12-22,19000.00,-3.43
34,S,ASK,24,XXX,SET,4500,31.25,25.25,-0.75,-2.88,0,2023-02-28,140625.00,-2.88
36,S,AWC,39,COS,SET50,9000,5.10,4.26,-0.10,-2.29,1,2022-12-22,45900.00,-2.29
39,S,JMART,15,5pct,SET50,1200,24.70,23.20,-0.80,-3.33,1,2023-08-24,29640.00,-3.33
42,S,PTG,17,5pct,SET100,3600,12.10,10.40,-0.30,-2.80,0,2022-12-22,43560.00,-2.80


In [24]:
df.query('percent >= 2.00')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
16,B,NER,-24,RD40pct,SET,3000,4.44,4.92,0.12,2.50,0,2023-08-25,13320.00,2.50
19,B,PTTEP,-56,XXX,SET50,200,134.50,162.50,4.00,2.52,0,2023-08-15,26900.00,2.52
27,B,STA,-8,RD65pct,SET100,2500,14.50,15.30,0.70,4.79,2,2023-08-24,36250.00,4.79


In [25]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False)

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
30,B,TOA,-2,5pct,SET,1000,26.00,26.50,0.00,0.00,2,2022-12-22,26000.00,0.00
7,B,GVREIT,-2,RD20pct,SET,6000,6.90,7.00,-0.05,-0.71,1,2023-08-28,41400.00,-0.71
2,B,BH,-5,XXX,SET50,100,250.00,255.00,-4.00,-1.54,1,2022-12-22,25000.00,-1.54
12,B,JASIF,-5,RD35pct,SET,4000,6.50,6.75,-0.05,-0.74,1,2023-08-21,26000.00,-0.74
31,B,TTB,-6,XXX,SET50,15000,1.64,1.70,-0.01,-0.58,1,2022-12-22,24600.00,-0.58
11,B,IVL,-6,RD30pct,SET50,800,27.50,29.00,0.00,0.00,1,2023-08-28,22000.00,0.00
15,B,M,-6,XXX,SET,500,46.00,47.50,0.00,0.00,1,2022-12-22,23000.00,0.00
14,B,KTB,-6,XXX,SET50,1000,18.80,19.40,0.10,0.52,1,2023-04-20,18800.00,0.52
26,B,SIRI,-7,XXX,SET,10000,1.90,1.97,-0.07,-3.43,1,2022-12-22,19000.00,-3.43
22,B,SAPPE,-8,XXX,SET,300,89.00,91.00,-9.00,-9.00,1,2022-12-22,26700.00,-9.00


### Sell recently purchased stocks if profitable

In [26]:
pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(107, 4)

In [27]:
colu = 'name date_x qty cost price difa %change %profit'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [28]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(106, 8)

In [29]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('name',ascending='False')

,name,date_x,qty,cost,price,difa,%change,%profit
95,TFFIF,2023-07-25,10000,7.45,7.50,500.00,-0.66,0.67


### Profit per Day

In [30]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2023 AND MONTH(sells.date) = 8
ORDER BY sells.date DESC, name


In [31]:
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days
0,JMART,2023,8,24.60,23.40,1.20,"1,200","29,520.00","28,080.00","1,440.00",5.13,171
1,WHART,2023,8,10.60,10.50,0.10,"10,000","106,000.00","105,000.00","1,000.00",0.95,471
2,JMART,2023,8,20.60,19.60,1.00,"1,200","24,720.00","23,520.00","1,200.00",5.10,126
3,ICHI,2023,8,15.90,14.30,1.60,"1,400","22,260.00","20,020.00","2,240.00",11.19,59
4,TTB,2023,8,1.76,1.64,0.12,"15,000","26,400.00","24,600.00","1,800.00",7.32,26
5,JMART,2023,8,18.30,17.00,1.30,"3,600","65,880.00","61,200.00","4,680.00",7.65,41
6,JMT,2023,8,43.50,39.75,3.75,"1,800","78,300.00","71,550.00","6,750.00",9.43,57


In [32]:
print(f"Monthly Profit Amount: {sells_df.gross.sum():,.2f}")

Monthly Profit Amount: 19,110.00


In [33]:
file_name = "monthly-sales.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [34]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day
0,JMART,2023,8,24.60,23.40,1.20,"1,200","29,520.00","28,080.00","1,440.00",5.13,171,0.029990
1,WHART,2023,8,10.60,10.50,0.10,"10,000","106,000.00","105,000.00","1,000.00",0.95,471,0.002022
2,JMART,2023,8,20.60,19.60,1.00,"1,200","24,720.00","23,520.00","1,200.00",5.10,126,0.040492
3,ICHI,2023,8,15.90,14.30,1.60,"1,400","22,260.00","20,020.00","2,240.00",11.19,59,0.189641
4,TTB,2023,8,1.76,1.64,0.12,"15,000","26,400.00","24,600.00","1,800.00",7.32,26,0.281426
5,JMART,2023,8,18.30,17.00,1.30,"3,600","65,880.00","61,200.00","4,680.00",7.65,41,0.186514
6,JMT,2023,8,43.50,39.75,3.75,"1,800","78,300.00","71,550.00","6,750.00",9.43,57,0.165508


In [35]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: 0.13%
